# River Sediment Data - Raw Data Import from Excel

The original dataset of water sediment samples was collected by Nez Perce-Clearwater National Forest personnel in northern Idaho, and stored, with sediment load calculations, in Excel.

This dataset includes more than three decades of sediment data for a single waterway location.

In this notebook, the raw data from the Excel spreadsheet is loaded into a Python pandas DataFrame, the structure is cleaned up to prepare for exploratory data analysis, and errors in loading are reported. 

**Output**:  Single CSV file including all years of data, ready to load into a pandas dataFrame in any other notebook.

Dataset processing and analysis by Laura Williams. 

NOTE:  Laura Williams is not a Forest Service employee, and the data processing and analysis are not necessarily endorsed by the US Forest Service.

In [1]:
# import packages
import pandas as pd
import numpy as np
import re
import pprint
from datetime import datetime

## Load raw data and check structure 
This code is set up so that additional years of data can be added to the Excel spreadsheet, and the updated spreadsheet can be loaded into this code and saved as a new CSV file for data analysis. 

In [2]:
raw_data = pd.ExcelFile('Data/Data_Sediment.xlsx')

***Assumption:*** In Excel, each yearly sheet is named with 4 digits

**CONFIRM the first and last year in the list below are as expected. The code will confirm if the years are consecutive.**

In [3]:
def get_year_list(raw_data):
    """Extract sheet names for all years.
    Output: list of years in raw data as text strings"""
    regex = re.compile(r'^\d\d\d\d$')
    year_list = [year for year in raw_data.sheet_names if regex.match(year)]
    return year_list

year_list = get_year_list(raw_data)

In [4]:
def check_years(year_list):
    """Check list of years to confirm if yearly Excel spreadsheeets are consecutive."""
    if (int(year_list[-1]) - int(year_list[0])) == (len(year_list) - 1):
        result = "All years between", year_list[0], "and", year_list[-1], 'are consecutive.'
    else:
        result = "Attention: not all years in the list are consecutive. Please review sheet names for accuracy."
    return result

print(check_years(year_list))
#Uncomment to print year_list
#print("\nFull list of years of data:")    
#pprint.pprint(year_list, compact = True)

('All years between', '1983', 'and', '2015', 'are consecutive.')


**Header set up**  
In these Excel spreadsheets, the 3rd row will be used as the header, while the 1st, 2nd, and 4th rows are not necessary for the header or for data analysis.  If additional years of data are added to this Excel file, the code below will confirm all headers are identical.

In [5]:
# Set index for header row
header_row = [2]
# Set index to not load second header row from Excel
skip_row = [3]

In [6]:
def get_header(data, year, header_row):
    """Extract header from an Excel sheet"""
    header = pd.read_excel(data, year, header=header_row, nrows=0)
    return header

In [7]:
def header_check(data, year_list, get_header, header_row):
    """Confirm all header rows in each sheet in an Excel workbook are identical
    
       Input: Excel workbook, spreadsheet names, get_header function, index of header row. 
       Output: Names of spreadsheets without identical headers
       """
       
    # Create header for first year
    header_year1 = get_header(data, year_list[0], header_row)    
    # Compare all subsequent headers to first header
    mismatch = []
    for sheet in range(len(year_list)-1):
        header = get_header(data, year_list[sheet+1], header_row)
        if header.equals(header_year1) == False:
            mismatch.append(year_list[sheet+1])
    return mismatch     

In [8]:
# Check header rows and report                           
if header_check(raw_data, year_list, get_header, header_row) == []:
    print("All spreadsheets have exactly the same header.")
else:
    print("Some years of data contain headers that do not match the header of the first year.")
    print("Please review and match headers before proceeding.")
    print("Spreadsheet years with headers not matching the first sheet of data:")
    print(mismatches)

All spreadsheets have exactly the same header.


## Set up data cleaning process, test on single year of data

### Identify columns to load from Excel worksheets
* Columns that define _how_ the data was collected or processed will not be used for data analysis. 
* The last 4 columns of each Excel spreadsheet define variable dropdown options and do not contain datapoint information.
* Empty columns that begin with "Unnamed" will not be loaded.
* **REVIEW the column names directly below to confirm they will not be used for data analysis.**

In [9]:
remove_cols = {'Bottle_No.',
               'Pan',
               'Nephelometric_Turbidity_Meter_Unit_Device',
               'Tubidity_Meter_Calibrated_(Yes/No)',
               'Tubidity_Meter_Calibrated_(If_No)',
               'Collection_Meth',
               'Collection_Meth_Calib',
               'Processing Method',
               'Analyst_Name',
               'Date_Processed_yyyy-dd-mm',
               'Notes',
               'Transferred_To_Masters_(Y/N)',
               'Nephelometric_Turbidity_Meter_Unit_Device.1',
               'Tubidity_Meter_Calibrated_(Yes/No).1',
               'Collection_Meth.1',
               'Processing Method.1',
               'Collection_Meth_Calib.1'}

In [10]:
def col_list(data, sheet_list, get_header, header_row):
    """Define list of columns to extract from Excel spreadsheet to use for data analysis."""
    all_cols = get_header(data, sheet_list[0], header_row).columns.tolist()
    use_cols = [i for i in all_cols if not re.match("Unnamed", i) if not (i in remove_cols)]
    return use_cols

use_cols = col_list(raw_data, year_list, get_header, header_row) 
print("These columns will be loaded from Excel into the dataset:")
pprint.pprint(use_cols)
print("\nTechnical Note: The 'Date Collected' column will be used as a row index instead of a dataset column.")

These columns will be loaded from Excel into the dataset:
['Date_Collected_yyyy-dd-mm',
 'Sample_Frequency_Hrs',
 'Filter_Tar_(B)_grams_(g)',
 'Filter_Sed_(A)_grams_(g)',
 'Sediment_grams_(g)',
 'Sediment_Calc_grams_(g)',
 'Sediment_Check_(Good/Bad)(±5%)',
 'Volume_Milliliters_(ml)',
 'Sedload_mg/l',
 'Sedload_Calc_mg/l',
 'Sedload_Check_(Good/Bad)(±5%)',
 'Nephelometric_Turbidity_Units_(NTU)']

Technical Note: The 'Date Collected' column will be used as a row index instead of a dataset column.


### Load data into pandas DataFrame

* Identify values in Excel spreadsheets to be assigned as True, False or NA
* Set up date as a datestamp row index

In [11]:
# assign variables for True, False, and NA
true_values = ['Good']
false_values = ['Bad', 'FALSE']
na_values = ['Hrs', 'grams_(g)', 'Good/Bad', 'milliliters_(ml)', 'mg/l', 'NTU']
# assign single year index variable for testing
single_year_index = 23

In [12]:
def load_year(raw_data, year_list, year_index, 
              header_row, use_cols, 
              true_values, false_values, na_values):
    """Input: Full raw data set as imported from Excel
              Functions and variables needed to isolate and process a single year of data.
              
       Output: single year of data with uniform format."""
    year_data = pd.read_excel(raw_data, 
                              year_list[year_index], 
                              header=header_row,
                              skiprows=skip_row,
                              usecols=use_cols, 
                              index_col=0,                              
                              true_values=true_values, 
                              false_values=false_values, 
                              na_values=na_values)
    year_data.index.name = 'Date_Collected_yyyy-mm-dd'
        
    return year_data

In [13]:
test_year = load_year(raw_data, year_list, single_year_index, 
                      header_row, use_cols, 
                      true_values, false_values, na_values)

In [14]:
print("Loading year", year_list[single_year_index])
print()
print("This year of data has", test_year.shape[0], "rows and", test_year.shape[1], "columns.")

test_year.head()

Loading year 2006

This year of data has 1011 rows and 11 columns.


,Sample_Frequency_Hrs,Filter_Tar_(B)_grams_(g),Filter_Sed_(A)_grams_(g),Sediment_grams_(g),Sediment_Calc_grams_(g),Sediment_Check_(Good/Bad)(±5%),Volume_Milliliters_(ml),Sedload_mg/l,Sedload_Calc_mg/l,Sedload_Check_(Good/Bad)(±5%),Nephelometric_Turbidity_Units_(NTU)
Date_Collected_yyyy-mm-dd,,,,,,,,,,,
2006-04-07,NaN,0.0815,0.0847,0.0032,0.0032,True,1020.0,5.88,3.137,False,NaN
2006-04-07,NaN,0.0824,0.0852,0.0028,0.0028,True,1020.0,5.88,2.745,False,NaN
2006-04-07,6.0,0.0842,0.0929,0.0087,0.0087,True,390.0,22.30,22.310,True,2.88
2006-04-07,6.0,0.0847,0.0914,0.0067,0.0067,True,530.0,12.60,12.640,True,2.75
2006-04-09,6.0,0.0842,0.0878,0.0036,0.0036,True,520.0,6.92,6.923,True,2.61


### Clean up data structure

In [15]:
def clean_year(year_data, year_list, year_index):
    """Input: Single year of data, identified by year_list and year_index
              
       Removes all rows without data points
       Checks for absence of data in spreadsheet
       Converts data types that do not always load into pandas with the correct data type
       Confirms all 'Date Collected' dates conform to sheet year name.
       Changes year in 'Date Collected' index if it does not match sheet year name
       Reports errors
       
       Output: The same year of data, cleaned, with errors reported.
    """    
    def fix_year_typos(year_data, year_list, year_index):
        """If year in 'Date Collected' index does not match year name of Excel spreadsheet:
           Assumption is made that year entry in Excel is a typo.
           The year in "Date Collected" is conformed to year in the sheet name.
        """
        new_index = [year_data.index[i].replace(year=int(year_list[year_index]))
                     for i in range(year_data.shape[0])]       
        year_data.index = new_index
        year_data.index.name = 'Date_Collected_yyyy-mm-dd'
        
        return year_data

    row_errors = []
    # Manage years without data - remove empty rows and check for remaining data points
    year_data = year_data[year_data.index.notnull()]    
    if year_data.shape[0] == 0:
        row_errors.extend(["Year", year_list[year_index], "has no data."])
    # Clean up data if it has been collected that year
    else:
        # convert index data type
        year_data.index = pd.to_datetime(year_data.index)
        # convert True/False column data types
        year_data = year_data.astype({'Sediment_Check_(Good/Bad)(±5%)': 'bool', 
                                      'Sedload_Check_(Good/Bad)(±5%)': 'bool'})
        # confirm/correct that all "Date Collected" dates match the sheet year name
        years_match = year_data.index.year == int(year_list[year_index])
        if years_match.mean() != 1:
            year_data = fix_year_typos(year_data, year_list, year_index)
            row_errors.extend(["At least one year value in 'Date Collected' entries in year", 
                               year_list[year_index], "does not match the year in the spreadsheet name.",
                               "The year has been corrected in the data. Please review original data."])
    
    return year_data, row_errors

In [16]:
test_year, row_errors = clean_year(test_year, year_list, single_year_index)

In [17]:
if type(test_year) == pd.DataFrame:
    print("This year of data now has", test_year.shape[0], "rows and", test_year.shape[1], "columns.")
else:
    for i in test_year:
        print(i)

This year of data now has 73 rows and 11 columns.


In [18]:
if type(test_year) == pd.DataFrame:
    print("Check data types:")
    print()
    print("IF there are more than 0 rows of data (data type doesn't matter for empty years):\n\
    Index data type should be datetime64[ns],\n\
    Sediment and Sedload Check columns should be bool,\n\
    Remainder of columns should be float64.\n\
    Actual data types are printed below:")
    print()
    print("Index data type:")
    print(test_year.index.dtype)
    print()
    print("Column data type:")
    print(test_year.dtypes)
else:
    for i in test_year:
        print(i)
        


Check data types:

IF there are more than 0 rows of data (data type doesn't matter for empty years):
    Index data type should be datetime64[ns],
    Sediment and Sedload Check columns should be bool,
    Remainder of columns should be float64.
    Actual data types are printed below:

Index data type:
datetime64[ns]

Column data type:
Sample_Frequency_Hrs                   float64
Filter_Tar_(B)_grams_(g)               float64
Filter_Sed_(A)_grams_(g)               float64
Sediment_grams_(g)                     float64
Sediment_Calc_grams_(g)                float64
Sediment_Check_(Good/Bad)(±5%)            bool
Volume_Milliliters_(ml)                float64
Sedload_mg/l                           float64
Sedload_Calc_mg/l                      float64
Sedload_Check_(Good/Bad)(±5%)             bool
Nephelometric_Turbidity_Units_(NTU)    float64
dtype: object


In [19]:
print("Errors from cleaning single year of data appear below (empty brackets reflect zero errors):")
pprint.pprint(row_errors)

Errors from cleaning single year of data appear below (empty brackets reflect zero errors):
["At least one year value in 'Date Collected' entries in year",
 '2006',
 'does not match the year in the spreadsheet name.',
 'The year has been corrected in the data. Please review original data.']


## CONCATENATE all years of data and report errors

In [20]:
def concat_all(raw_data, year_list,
              load_year, clean_year, 
              header_row, use_cols, 
              true_values, false_values, na_values):
    """Load and clean and all years of data from the Excel spreadsheet.
       Output: pandas DataFrame containing all years of cleaned-up sediment data."""
    # initialize error variable
    row_errors_all = []
    # load, clean and concatentate one year at a time
    for year_index in range(len(year_list)):
        year_data = load_year(raw_data, year_list, year_index, 
                              header_row, use_cols, 
                              true_values, false_values, na_values)
        if type(year_data) == pd.DataFrame and year_data.shape[0] > 0: 
            year_clean, row_errors = clean_year(year_data, year_list, year_index)
            if year_index == 0:
                sed_data = year_clean
            else:
                sed_data = pd.concat([sed_data, year_clean])
        row_errors_all.extend(row_errors)
        
    # Because the boolean data types still sometimes don't convert properly
    sed_data = sed_data.astype({'Sediment_Check_(Good/Bad)(±5%)': 'bool', 
                                'Sedload_Check_(Good/Bad)(±5%)': 'bool'})
        
    return sed_data, row_errors_all

In [21]:
sed_data, row_errors_all = concat_all(raw_data, year_list,
                                      load_year, clean_year, 
                                      header_row, use_cols, 
                                      true_values, false_values, na_values)

In [22]:
print("Errors from cleaning all years:")
pprint.pprint(row_errors_all)

Errors from cleaning all years:
['Year',
 '1991',
 'has no data.',
 "At least one year value in 'Date Collected' entries in year",
 '2006',
 'does not match the year in the spreadsheet name.',
 'The year has been corrected in the data. Please review original data.']


### Check  structure of full dataset

In [23]:
print("Full dataset has", sed_data.shape[0], "rows and", sed_data.shape[1], "columns.")

Full dataset has 2697 rows and 11 columns.


In [24]:
print("Columns in full dataset:")
for name in sed_data.columns:
    print(name)

Columns in full dataset:
Sample_Frequency_Hrs
Filter_Tar_(B)_grams_(g)
Filter_Sed_(A)_grams_(g)
Sediment_grams_(g)
Sediment_Calc_grams_(g)
Sediment_Check_(Good/Bad)(±5%)
Volume_Milliliters_(ml)
Sedload_mg/l
Sedload_Calc_mg/l
Sedload_Check_(Good/Bad)(±5%)
Nephelometric_Turbidity_Units_(NTU)


In [25]:
print("Years included in full dataset.")
print(sed_data.index.year.unique())
print()
print("NOTE: 1991 is not in the final dataset because there was no data for that year in the Excel spreadhseet")

Years included in full dataset.
Int64Index([1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1992, 1993, 1994,
            1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
            2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015],
           dtype='int64', name='Date_Collected_yyyy-mm-dd')

NOTE: 1991 is not in the final dataset because there was no data for that year in the Excel spreadhseet


In [26]:
print("Check data types in concatenated dataset:")
print()
print("Index data type should be datetime64[ns],\n\
    Sediment and Sedload Check columns should be bool,\n\
    remainder of columns should be float64.\n\
    Actual data types are printed below:")
print()
print("Index data type:")
print(sed_data.index.dtype)
print()
print("Column data type:")
print(sed_data.dtypes)

Check data types in concatenated dataset:

Index data type should be datetime64[ns],
    Sediment and Sedload Check columns should be bool,
    remainder of columns should be float64.
    Actual data types are printed below:

Index data type:
datetime64[ns]

Column data type:
Sample_Frequency_Hrs                   float64
Filter_Tar_(B)_grams_(g)               float64
Filter_Sed_(A)_grams_(g)               float64
Sediment_grams_(g)                     float64
Sediment_Calc_grams_(g)                float64
Sediment_Check_(Good/Bad)(±5%)            bool
Volume_Milliliters_(ml)                float64
Sedload_mg/l                           float64
Sedload_Calc_mg/l                      float64
Sedload_Check_(Good/Bad)(±5%)             bool
Nephelometric_Turbidity_Units_(NTU)    float64
dtype: object


In [27]:
sed_data.head()

,Sample_Frequency_Hrs,Filter_Tar_(B)_grams_(g),Filter_Sed_(A)_grams_(g),Sediment_grams_(g),Sediment_Calc_grams_(g),Sediment_Check_(Good/Bad)(±5%),Volume_Milliliters_(ml),Sedload_mg/l,Sedload_Calc_mg/l,Sedload_Check_(Good/Bad)(±5%),Nephelometric_Turbidity_Units_(NTU)
Date_Collected_yyyy-mm-dd,,,,,,,,,,,
1983-04-01,NaN,0.1104,0.1108,0.0004,0.0004,True,339.0,1.1799,1.1800,True,NaN
1983-04-02,NaN,0.1119,0.1123,0.0004,0.0004,True,337.0,1.1869,1.1870,True,NaN
1983-04-03,NaN,0.1118,0.1121,0.0003,0.0003,True,338.0,0.8876,0.8876,True,NaN
1983-04-04,NaN,0.1106,0.1185,0.0079,0.0079,True,185.0,42.7027,42.7000,True,NaN
1983-04-05,NaN,0.1089,0.1099,0.0010,0.0010,True,206.0,4.8544,4.8540,True,NaN


In [28]:
# display a random slice of dataset
sed_data[1000:1005]

,Sample_Frequency_Hrs,Filter_Tar_(B)_grams_(g),Filter_Sed_(A)_grams_(g),Sediment_grams_(g),Sediment_Calc_grams_(g),Sediment_Check_(Good/Bad)(±5%),Volume_Milliliters_(ml),Sedload_mg/l,Sedload_Calc_mg/l,Sedload_Check_(Good/Bad)(±5%),Nephelometric_Turbidity_Units_(NTU)
Date_Collected_yyyy-mm-dd,,,,,,,,,,,
1995-06-03,6.0,0.0519,0.0623,0.0104,0.0104,True,261.0,39.8467,39.850,True,5.5
1995-06-06,6.0,0.0721,0.0757,0.0036,0.0036,True,465.0,7.7490,7.742,True,1.4
1995-06-06,6.0,0.0782,0.0789,0.0007,0.0007,True,298.0,2.3490,2.349,True,2.3
1995-06-07,6.0,0.0853,0.0926,0.0073,0.0073,True,403.0,18.1141,18.110,True,1.5
1995-06-08,6.0,0.0510,0.0533,0.0023,0.0023,True,423.0,5.4374,5.437,True,1.6


## Save concatenated, cleaned dataset to CSV file
**IMPORTANT**  
This code will overwrite any existing CSV file with the same name.  The filename below has been changed to prevent overwriting the dataset currently in use.  

In [29]:
filename = 'Data/sed_data_full_test.csv'
sed_data.to_csv(filename)

### Optional - test that data reloads properly

Compare number of rows and columns, content of first few rows, and datatypes to original above

In [30]:
test_data = pd.read_csv(filename, 
                        encoding='UTF-8', 
                        index_col='Date_Collected_yyyy-mm-dd',
                        parse_dates = ['Date_Collected_yyyy-mm-dd']
                       )

In [31]:
print("Rows:", test_data.shape[0], "Columns:", test_data.shape[1])

Rows: 2697 Columns: 11


In [32]:
test_data.head(5)

,Sample_Frequency_Hrs,Filter_Tar_(B)_grams_(g),Filter_Sed_(A)_grams_(g),Sediment_grams_(g),Sediment_Calc_grams_(g),Sediment_Check_(Good/Bad)(±5%),Volume_Milliliters_(ml),Sedload_mg/l,Sedload_Calc_mg/l,Sedload_Check_(Good/Bad)(±5%),Nephelometric_Turbidity_Units_(NTU)
Date_Collected_yyyy-mm-dd,,,,,,,,,,,
1983-04-01,NaN,0.1104,0.1108,0.0004,0.0004,True,339.0,1.1799,1.1800,True,NaN
1983-04-02,NaN,0.1119,0.1123,0.0004,0.0004,True,337.0,1.1869,1.1870,True,NaN
1983-04-03,NaN,0.1118,0.1121,0.0003,0.0003,True,338.0,0.8876,0.8876,True,NaN
1983-04-04,NaN,0.1106,0.1185,0.0079,0.0079,True,185.0,42.7027,42.7000,True,NaN
1983-04-05,NaN,0.1089,0.1099,0.0010,0.0010,True,206.0,4.8544,4.8540,True,NaN


In [33]:
print("Compare data types of reloaded data to original data above\n")
print("Index data type:")
print(sed_data.index.dtype, '\n')
print("Column data type:")
print(sed_data.dtypes)

Compare data types of reloaded data to original data above

Index data type:
datetime64[ns] 

Column data type:
Sample_Frequency_Hrs                   float64
Filter_Tar_(B)_grams_(g)               float64
Filter_Sed_(A)_grams_(g)               float64
Sediment_grams_(g)                     float64
Sediment_Calc_grams_(g)                float64
Sediment_Check_(Good/Bad)(±5%)            bool
Volume_Milliliters_(ml)                float64
Sedload_mg/l                           float64
Sedload_Calc_mg/l                      float64
Sedload_Check_(Good/Bad)(±5%)             bool
Nephelometric_Turbidity_Units_(NTU)    float64
dtype: object


**END OF DATA CLEAN UP**